In [ ]:
!pip install scikeras

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, SpatialDropout1D

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load your dataset
spam_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI Final Project Work/combined_data.csv")

In [ ]:
labels = spam_data["label"]
messages = spam_data["text"]

In [ ]:
# Tokenize the messages
tokenizer = Tokenizer()
tokenizer.fit_on_texts(messages)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Convert messages to sequences
sequences = tokenizer.texts_to_sequences(messages)

In [ ]:
# Paddng and truncating all sequences to the same length
max_sequence_len = 30
padded_sequences = pad_sequences(sequences, max_sequence_len)

In [ ]:
X_data = padded_sequences
Y_data = labels

In [ ]:
# Splitting dataset: 20% trainnig 80% testing
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 42)

In [ ]:
# Number of categories (Span or Not Spam)
num_classes = 2

In [ ]:
# Size of vector representation of each word
embedding_dim = 100

In [ ]:
# Defining the Bidirectional model
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = embedding_dim, input_length = max_sequence_len))
model.add(Bidirectional(LSTM(units = 64, return_sequences = True)))
model.add(LSTM(units = 32, return_sequences = True))
model.add(LSTM(units = 16))
model.add(Dense(units = num_classes, activation = "sigmoid"))  # Changed to softmax for multi-class classification

In [ ]:
# Compile the model
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_78"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_78 (Embedding)    (None, 30, 100)           32358500  
                                                                 
 bidirectional_78 (Bidirect  (None, 30, 128)           84480     
 ional)                                                          
                                                                 
 lstm_235 (LSTM)             (None, 30, 32)            20608     
                                                                 
 lstm_236 (LSTM)             (None, 16)                3136      
                                                                 
 dense_78 (Dense)            (None, 2)                 34        
                                                                 
Total params: 32466758 (123.85 MB)
Trainable params: 32466758 (123.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________

In [ ]:
# Convert labels to categorical format
Y_train_categorical = to_categorical(Y_train, num_classes)
Y_test_categorical = to_categorical(Y_test, num_classes)

In [ ]:
# Training the model
BD_model = model.fit(X_train, Y_train_categorical, epochs = 1, batch_size = 32, validation_data = (X_test, Y_test_categorical))

2087/2087 [==============================] - 1956s 933ms/step - loss: 0.1086 - accuracy: 0.9598 - val_loss: 0.0629 - val_accuracy: 0.9772


In [ ]:
def create_model(units = 32, dropout_rate = 0.2):
    model = Sequential()
    model.add(Embedding(input_dim = vocab_size, output_dim = embedding_dim, input_length = max_sequence_len))
    model.add(Bidirectional(LSTM(units = 64, return_sequences = True)))
    model.add(LSTM(units = 32, return_sequences = True))
    model.add(LSTM(units = 16))
    model.add(Dense(units = num_classes, activation = "sigmoid"))
    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    return model

In [ ]:
model = KerasClassifier(build_fn = create_model, epochs = 1, batch_size = 32, verbose = 1, units = 32, dropout_rate = 0.2)

In [ ]:
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf_keras', model)
])

In [ ]:
# Define the hyperparameter grid
param_grid = {
    "units": [32, 64],
    "dropout_rate": [0.2, 0.4]
}

In [ ]:
# Create StratifiedKFold object
grid_search = GridSearchCV(estimator = model , param_grid = param_grid, scoring = "accuracy", cv = 3, verbose = 1)

In [ ]:
# Create GridSearchCV object
grid_result = grid_search.fit(X_train, Y_train_categorical)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


696/696 [==============================] - 17s 22ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


696/696 [==============================] - 17s 22ms/step


...

696/696 [==============================] - 17s 23ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


696/696 [==============================] - 18s 24ms/step


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


1409/2087 [===================>..........] - ETA: 10:55 - loss: 0.1236 - accuracy: 0.9543

In [ ]:
# Get the best parameters
best_parameters = grid_result.best_params_
best_parameters

In [ ]:

Y_pred = spam_model.predict(X_test)
accuracy = accuracy_score(Y_test_categorical, Y_pred)
print(f"Best parameters: {best_parameters}")
print(f"Accuracy test: {accuracy}")

# ALTERNATIVE
# accuracy = grid_result.score(X_test, Y_test)
# print(f"Best parameters: {best_parameters}")
# print(f"Accuracy test: {accuracy}")

In [ ]:
spam_model = grid_result.best_estimator_

In [ ]:
spam_model.fit(X_train, Y_train, validation_data = (X_test, Y_test))

# ALTERNATIVE
# Delete the code above

In [ ]:
# Save the entire model to a file
model.save("/content/drive/MyDrive/Colab Notebooks/AI Final Project Work/spam_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Saving the trained model
model.save("spam_model.h5")